In [3]:
import pandas as pd

## Erstelle aus `bank-churn.csv` einen DataFrame

In [4]:
data = pd.read_csv('bank-churn.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Data Cleaning


In [10]:
data.corr()['Exited'].apply(lambda x: abs(x)).sort_values(ascending=False)

Exited             1.000000
Age                0.285323
IsActiveMember     0.156128
Balance            0.118533
NumOfProducts      0.047820
CreditScore        0.027094
RowNumber          0.016571
Tenure             0.014001
EstimatedSalary    0.012097
HasCrCard          0.007138
CustomerId         0.006248
Name: Exited, dtype: float64

In [39]:
data = data.drop(["RowNumber", "CustomerId", "Surname"], axis=1)

In [40]:
data = pd.get_dummies(data=data, columns=["Geography", "Gender"], drop_first=True)
data.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1,0


## Training
* 20% Testdaten
* `random_state=42` sofern unterstützt
* target feature: `Exited`
* Ziel: score von 0.85

In [41]:
from sklearn.model_selection import train_test_split

RANDOM_STATE = 42

X = data.drop(["Exited"], axis=1)
y = data["Exited"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

In [42]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [43]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

search = GridSearchCV(estimator=SVC(),
                      param_grid={'C': [0.1, 1, 10],
                                  'gamma': [0.1, 0.01, 0.001],
                                  'class_weight': [None, "balanced"]},
                      n_jobs=-1)
search.fit(X_train, y_train)
print("Best params:" , search.best_params_)

Best params: {'C': 1, 'class_weight': None, 'gamma': 0.1}


In [56]:
best_C= search.best_params_['C']
best_gamma= search.best_params_['gamma']
best_class_weight= search.best_params_['class_weight']
model = SVC(C=best_C, gamma=best_gamma, class_weight=best_class_weight, probability=True)
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.856

## Interpretiere den rechten oberen Eintrag der Confusion Matrix

In [45]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, model.predict(X_test))

array([[1561,   46],
       [ 242,  151]], dtype=int64)

KI sagt, person steigt nicht aus, aber person steigt aus

## Predicte, mit welcher Wahrscheinlichkeit der Kunde aussteigen wird
Er sollte mit etwa 90%iger Wahrscheinlichkeit **bleiben**

In [57]:
customer = pd.DataFrame(data={
    'CreditScore': 500,
    'Age': 20,
    'Tenure': 2,
    'Balance': 0,
    'NumOfProducts': 1,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 60_000,
    'Geography': 'Germany',
    'Gender': 'Male'},
    index=[0])

custom_customer = pd.DataFrame(data={
    'CreditScore': 500,
    'Age': 20,
    'Tenure': 2,
    'Balance': 0,
    'NumOfProducts': 1,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 60_000,
    'Geography_Germany': 1,
    'Geography_Spain': 0,
    'Gender_Male': 1},
    index=[0])

custom_customer_scaled = scaler.transform(custom_customer)
model.predict_proba(custom_customer_scaled)

array([[0.92253001, 0.07746999]])